In [2]:
import string
import random
import tkinter as tk
from tkinter import scrolledtext, font, messagebox, ttk
from collections import defaultdict
from datetime import datetime
import json
import os
import time
import threading

# Force PyTorch backend
os.environ["USE_TF"] = "0"
os.environ["USE_TORCH"] = "1"

from sentence_transformers import SentenceTransformer, util

# ------------------- Enhanced Configuration -------------------
CONFIG = {
    "hotel_name": "Grand Horizon Hotel",
    "bot_name": "Horizon Concierge",
    "theme_colors": {
        "primary": "#2c3e50",       # Dark blue
        "secondary": "#3498db",      # Bright blue
        "accent": "#e74c3c",         # Red
        "background": "#f5f7fa",     # Light gray
        "text": "#2c3e50",           # Dark text
        "light_text": "#ffffff",     # White text
        "user_bubble": "#e3f2fd",    # Light blue
        "bot_bubble": "#f0f0f0",     # Light gray
        "border": "#cfd8dc"          # Gray border
    },
    "font_family": "Segoe UI",
    "title_font_size": 16,
    "chat_font_size": 12,
    "button_font_size": 11,
    "suggestion_font_size": 10,
    "window_size": "900x700",
    "chat_area_height": 25,
    "typing_speed": 0.02,
    "thinking_delay": (0.8, 1.8),
    "default_suggestions": [          # Added default suggestions to prevent KeyError
        "How's the staff service?",
        "Is the location convenient?",
        "Tell me about food options",
        "How clean are the rooms?",
        "What facilities do you have?"
    ]
}

# ------------------- Model Loader -------------------
class ModelLoader:
    def __init__(self):
        self.model = None
        self.loaded = False
        
    def load_model(self):
        if not self.loaded:
            self.model = SentenceTransformer("all-MiniLM-L6-v2")
            self.loaded = True

model_loader = ModelLoader()

# ------------------- Knowledge Base -------------------
knowledge_base = {
    "staff": {
        "responses": [
            "🌟 Our 24/7 staff receives regular hospitality training. Recent guests praised their attentiveness in 89% of reviews.",
            "👔 The multilingual concierge team can assist with reservations, directions, and special requests with an average 4.8/5 satisfaction rating.",
            "🧹 Our housekeeping team follows a 50-point cleaning checklist for each room, with supervisors conducting random quality checks."
        ],
        "keywords": ["staff", "service", "reception", "concierge", "employee", "manager"],
        "follow_up": [
            "What languages do your staff speak?",
            "How quickly does housekeeping respond to requests?",
            "Can I request a specific staff member?"
        ]
    },
    "cleanliness": {
        "responses": [
            "🧼 We follow CDC cleaning guidelines with hospital-grade disinfectants. All rooms receive deep cleaning between guests.",
            "✨ Our housekeeping staff performs 25-point bathroom inspections and uses UV lights to verify cleanliness.",
            "🛏️ Linens are changed daily and laundered at 160°F to ensure complete sanitation."
        ],
        "keywords": ["clean", "dirty", "hygiene", "sanitize", "housekeeping", "tidy"],
        "follow_up": [
            "How often are rooms cleaned?",
            "Do you use eco-friendly cleaning products?",
            "What's your COVID-19 cleaning protocol?"
        ]
    },
    "location": {
        "responses": [
            "📍 We're located in the heart of downtown, just 300m from the central subway station and 5 minutes walk to major attractions.",
            "🌆 The hotel offers panoramic city views and is adjacent to the business district with easy access to 3 metro lines."
        ],
        "keywords": ["location", "near", "distance", "transport", "metro", "walk"],
        "follow_up": [
            "Is there a shuttle service from the airport?",
            "What restaurants are nearby?",
            "How safe is the neighborhood at night?"
        ]
    },
    "food": {
        "responses": [
            "🍳 Our breakfast buffet features 50+ items including local specialties. Room service is available 6AM-11PM.",
            "🍽️ The rooftop restaurant serves international cuisine with seasonal menus, accommodating all dietary needs."
        ],
        "keywords": ["food", "restaurant", "breakfast", "dining", "meal", "menu"],
        "follow_up": [
            "Do you have vegan options?",
            "What are the breakfast hours?",
            "Is there a dress code for the restaurant?"
        ]
    },
    "room": {
        "responses": [
            "🛌 Rooms feature premium Simmons mattresses, blackout curtains, and soundproofing with 35dB reduction.",
            "📺 All rooms include 55\" smart TVs, mini-fridges, and luxury toiletries. Suites add sitting areas or balconies."
        ],
        "keywords": ["room", "bed", "view", "quiet", "noise", "amenities"],
        "follow_up": [
            "What room categories do you offer?",
            "Do rooms have coffee makers?",
            "Is there a minibar in the rooms?"
        ]
    },
    "facilities": {
        "responses": [
            "🏊 Our facilities include a heated indoor pool (open 6AM-10PM), 24-hour fitness center, and business lounge.",
            "💆 The spa offers massages, facials, and a sauna. Complimentary high-speed WiFi (100Mbps) throughout."
        ],
        "keywords": ["pool", "gym", "spa", "wifi", "internet", "facility"],
        "follow_up": [
            "What are the pool hours?",
            "Is there a sauna or steam room?",
            "Do you have a business center?"
        ]
    },
    "booking": {
        "responses": [
            "📅 Book directly on our website for best rates. Flexible cancellation up to 48 hours before arrival.",
            "💳 We offer seasonal packages with breakfast or spa credits. Group rates available for 10+ rooms."
        ],
        "keywords": ["book", "reservation", "rate", "price", "cancel", "check-in"],
        "follow_up": [
            "What's your cancellation policy?",
            "Do you offer long-stay discounts?",
            "Can I modify my reservation?"
        ]
    }
}

# General questions
general_questions = {
    "What are your check-in/check-out times?": {
        "answer": "⏰ Check-in is at 3:00 PM and check-out is at 11:00 AM. Early check-in from 1PM ($25 fee) and late check-out until 2PM ($35 fee) may be available.",
        "follow_up": [
            "Is early check-in guaranteed?",
            "What's the fee for late check-out?"
        ]
    },
    "Do you have parking?": {
        "answer": "🚗 We offer valet parking ($35/night) and self-parking ($25/night). EV charging stations available at no extra cost.",
        "follow_up": [
            "Is parking included in the room rate?",
            "How many parking spaces are available?"
        ]
    },
    "What's your cancellation policy?": {
        "answer": "❌ Standard reservations can be cancelled 48 hours prior without penalty. Non-refundable rates save 15% but cannot be changed.",
        "follow_up": [
            "What about special event periods?",
            "Can I get a refund if I leave early?"
        ]
    }
}

# ------------------- Conversation Management -------------------
class Conversation:
    def __init__(self):
        self.context = {"user_name": None}
        self.history = []
        
    def add_interaction(self, user_input, bot_response):
        self.history.append({
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "user": user_input,
            "bot": bot_response
        })
        
    def get_context(self, key):
        return self.context.get(key)
        
    def set_context(self, key, value):
        self.context[key] = value
        
    def save_history(self):
        try:
            with open("chat_history.json", "a") as f:
                json.dump(self.history[-10:], f)  # Save last 10 messages
                f.write("\n")
        except Exception as e:
            print(f"Error saving history: {e}")

# ------------------- Enhanced Response Generation -------------------
def get_response(user_input, conversation):
    user_input_lower = user_input.lower().strip()
    
    # Check for name
    if not conversation.get_context("user_name"):
        name_keywords = ["my name is", "i'm", "i am"]
        for phrase in name_keywords:
            if phrase in user_input_lower:
                name = user_input[user_input.lower().find(phrase)+len(phrase):].strip()
                conversation.set_context("user_name", name.split()[0].title())
                return f"Nice to meet you, {conversation.get_context('user_name')}! How can I assist you today?", []
    
    # Greetings
    greeting_responses = {
        "hi": ["👋 Hello! Welcome to {hotel}! How can I help you today?",
               "👋 Hi there! I'm {bot}, your virtual concierge."],
        "hello": ["👋 Greetings! How may I help you with your stay at {hotel}?",
                 "👋 Hello! Ready to assist with anything about our hotel."],
        "hey": ["👋 Hey there! What would you like to know about {hotel}?",
                "👋 Hey! Ask me anything about our hotel services."],
        "bye": ["👋 Goodbye! We hope to welcome you soon at {hotel}!",
               "👋 Have a wonderful day!"],
        "goodbye": ["👋 Thank you for chatting with us. Safe travels!", 
                   "👋 Farewell! We look forward to serving you."],
        "thanks": ["🙏 You're welcome! Is there anything else I can help with?", 
                 "🙏 My pleasure!"],
        "thank you": ["🙏 Happy to help! Enjoy your stay with us.", 
                     "🙏 You're very welcome!"],
        "how are you": ["🤖 I'm just a bot, but I'm functioning well! How can I assist you today?",
                       "🤖 Always happy to help guests like you!"],
        "what's up": ["🌟 Ready to help with your {hotel} questions!",
                     "🌟 All systems go! How can I assist you today?"]
    }
    
    for greeting in greeting_responses:
        if greeting in user_input_lower:
            response = random.choice(greeting_responses[greeting])
            return response.format(
                hotel=CONFIG["hotel_name"],
                bot=CONFIG["bot_name"]
            ), []
    
    # Check category responses
    for cat, data in knowledge_base.items():
        if any(keyword in user_input_lower for keyword in data["keywords"]):
            return random.choice(data["responses"]), data["follow_up"]
    
    # Check general questions
    for question, data in general_questions.items():
        if any(word in user_input_lower for word in question.lower().split()):
            return data["answer"], data["follow_up"]
    
    # Fallback with default suggestions
    return "🤔 I'm not sure about that. Here are some things I can help with:", CONFIG["default_suggestions"]

# ------------------- Advanced Tkinter GUI -------------------
class HotelChatbotApp:
    def __init__(self, root):
        self.root = root
        self.typing = False
        self.conversation = Conversation()
        self.setup_window()
        self.create_widgets()
        self.display_welcome_message()
        
    def setup_window(self):
        self.root.title(f"{CONFIG['hotel_name']} - {CONFIG['bot_name']}")
        self.root.geometry(CONFIG["window_size"])
        self.root.configure(bg=CONFIG["theme_colors"]["background"])
        self.root.minsize(800, 600)
        
        # Configure grid layout
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=1)
        
    def create_widgets(self):
        # Header Frame
        self.header_frame = tk.Frame(
            self.root,
            bg=CONFIG["theme_colors"]["primary"],
            height=60,
            padx=20
        )
        self.header_frame.grid(row=0, column=0, sticky="ew")
        
        self.title_label = tk.Label(
            self.header_frame,
            text=f"{CONFIG['hotel_name']} Concierge",
            font=(CONFIG["font_family"], CONFIG["title_font_size"], "bold"),
            bg=CONFIG["theme_colors"]["primary"],
            fg=CONFIG["theme_colors"]["light_text"]
        )
        self.title_label.pack(side=tk.LEFT)
        
        # Chat Frame
        self.chat_frame = tk.Frame(
            self.root,
            bg=CONFIG["theme_colors"]["background"]
        )
        self.chat_frame.grid(row=1, column=0, sticky="nsew", padx=20, pady=(0, 10))
        self.chat_frame.grid_rowconfigure(0, weight=1)
        self.chat_frame.grid_columnconfigure(0, weight=1)
        
        # Chat Text Widget with Custom Styles
        self.chat_text = tk.Text(
            self.chat_frame,
            wrap=tk.WORD,
            font=(CONFIG["font_family"], CONFIG["chat_font_size"]),
            bg="white",
            padx=15,
            pady=10,
            state=tk.DISABLED,
            relief=tk.FLAT,
            borderwidth=0
        )
        self.chat_text.grid(row=0, column=0, sticky="nsew")
        
        # Scrollbar
        self.scrollbar = ttk.Scrollbar(
            self.chat_frame,
            orient=tk.VERTICAL,
            command=self.chat_text.yview
        )
        self.scrollbar.grid(row=0, column=1, sticky="ns")
        self.chat_text.config(yscrollcommand=self.scrollbar.set)
        
        # Typing Indicator
        self.typing_label = tk.Label(
            self.chat_frame,
            text="",
            font=(CONFIG["font_family"], CONFIG["chat_font_size"]-1, "italic"),
            bg=CONFIG["theme_colors"]["background"],
            fg=CONFIG["theme_colors"]["text"]
        )
        self.typing_label.grid(row=1, column=0, sticky="w", padx=5)
        
        # Suggestions Frame
        self.suggestion_frame = tk.Frame(
            self.root,
            bg=CONFIG["theme_colors"]["background"],
            height=40
        )
        self.suggestion_frame.grid(row=2, column=0, sticky="ew", padx=20)
        
        # Input Frame
        self.input_frame = tk.Frame(
            self.root,
            bg=CONFIG["theme_colors"]["background"],
            padx=10,
            pady=10
        )
        self.input_frame.grid(row=3, column=0, sticky="ew")
        
        self.input_entry = tk.Entry(
            self.input_frame,
            font=(CONFIG["font_family"], CONFIG["chat_font_size"]),
            bg="white",
            relief=tk.FLAT,
            borderwidth=2
        )
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=(0, 10))
        self.input_entry.bind("<Return>", lambda e: self.send_message())
        
        self.send_button = tk.Button(
            self.input_frame,
            text="Send",
            command=self.send_message,
            font=(CONFIG["font_family"], CONFIG["button_font_size"], "bold"),
            bg=CONFIG["theme_colors"]["secondary"],
            fg=CONFIG["theme_colors"]["light_text"],
            activebackground=CONFIG["theme_colors"]["accent"],
            relief=tk.FLAT,
            padx=20
        )
        self.send_button.pack(side=tk.RIGHT)
        
        # Configure text tags
        self.configure_text_tags()
    
    def configure_text_tags(self):
        # User message styling
        self.chat_text.tag_config(
            "user",
            foreground="#2c3e50",
            font=(CONFIG["font_family"], CONFIG["chat_font_size"], "bold"),
            lmargin1=50,
            lmargin2=50,
            rmargin=50,
            spacing3=5
        )
        
        # Bot message styling
        self.chat_text.tag_config(
            "bot",
            foreground="#2c3e50",
            font=(CONFIG["font_family"], CONFIG["chat_font_size"]),
            lmargin1=50,
            lmargin2=50,
            rmargin=50,
            spacing3=5
        )
        
        # Timestamp styling
        self.chat_text.tag_config(
            "timestamp",
            foreground="#7f8c8d",
            font=(CONFIG["font_family"], CONFIG["chat_font_size"]-2)
        )
    
    def display_welcome_message(self):
        welcome_msg = (
            f"🌟 Welcome to {CONFIG['hotel_name']}! 🌟\n\n"
            f"I'm {CONFIG['bot_name']}, your virtual concierge assistant.\n\n"
            "I can help you with:\n"
            "• Room reservations and upgrades\n• Dining options and hours\n"
            "• Spa and facility information\n• Local attractions and transportation\n"
            "• Any other hotel services\n\n"
            "How may I assist you today?"
        )
        self.display_bot_message(welcome_msg)
        self.update_suggestions(CONFIG["default_suggestions"])
    
    def send_message(self):
        user_input = self.input_entry.get().strip()
        if not user_input:
            return
            
        self.display_user_message(user_input)
        self.input_entry.delete(0, tk.END)
        
        threading.Thread(
            target=self.process_response,
            args=(user_input,),
            daemon=True
        ).start()
    
    def process_response(self, user_input):
        self.show_typing(True)
        time.sleep(random.uniform(*CONFIG["thinking_delay"]))
        
        response, follow_up = get_response(user_input, self.conversation)
        self.conversation.add_interaction(user_input, response)
        
        self.display_bot_message(response, typing_effect=True)
        self.show_typing(False)
        
        self.update_suggestions(follow_up if follow_up else CONFIG["default_suggestions"])
    
    def show_typing(self, show):
        self.typing = show
        if show:
            self.typing_label.config(text=f"{CONFIG['bot_name']} is typing...")
        else:
            self.typing_label.config(text="")
    
    def display_user_message(self, message):
        self.chat_text.config(state=tk.NORMAL)
        timestamp = datetime.now().strftime("%H:%M")
        self.chat_text.insert(tk.END, f"[{timestamp}] ", "timestamp")
        self.chat_text.insert(tk.END, "You: ", "user")
        self.chat_text.insert(tk.END, message + "\n", "user")
        self.chat_text.config(state=tk.DISABLED)
        self.chat_text.see(tk.END)
    
    def display_bot_message(self, message, typing_effect=False):
        self.chat_text.config(state=tk.NORMAL)
        timestamp = datetime.now().strftime("%H:%M")
        self.chat_text.insert(tk.END, f"[{timestamp}] ", "timestamp")
        self.chat_text.insert(tk.END, f"{CONFIG['bot_name']}: ", "bot")
        
        if typing_effect:
            self.chat_text.config(state=tk.NORMAL)
            self.chat_text.update()
            for char in message:
                if not self.typing:
                    break
                self.chat_text.insert(tk.END, char)
                self.chat_text.see(tk.END)
                time.sleep(CONFIG["typing_speed"])
        else:
            self.chat_text.insert(tk.END, message)
        
        self.chat_text.insert(tk.END, "\n\n")
        self.chat_text.config(state=tk.DISABLED)
        self.chat_text.see(tk.END)
    
    def update_suggestions(self, suggestions):
        for widget in self.suggestion_frame.winfo_children():
            widget.destroy()
        
        for i, suggestion in enumerate(suggestions[:4]):
            btn = tk.Button(
                self.suggestion_frame,
                text=suggestion,
                command=lambda s=suggestion: self.use_suggestion(s),
                font=(CONFIG["font_family"], CONFIG["suggestion_font_size"]),
                bg=CONFIG["theme_colors"]["background"],
                fg=CONFIG["theme_colors"]["secondary"],
                activeforeground=CONFIG["theme_colors"]["accent"],
                relief=tk.FLAT,
                borderwidth=0,
                padx=10,
                pady=2
            )
            btn.pack(side=tk.LEFT, padx=5)
    
    def use_suggestion(self, suggestion):
        self.input_entry.delete(0, tk.END)
        self.input_entry.insert(0, suggestion)
        self.input_entry.focus_set()

# ------------------- Main Application -------------------
def main():
    root = tk.Tk()
    
    # Set window icon if available
    try:
        root.iconbitmap("hotel_icon.ico")
    except:
        pass
    
    # Load model in background
    threading.Thread(target=model_loader.load_model, daemon=True).start()
    
    # Create and run app
    app = HotelChatbotApp(root)
    root.mainloop()
    
    # Save conversation history on exit
    app.conversation.save_history()

if __name__ == "__main__":
    main() 